In [11]:
# 拉勾网静态数据采集
import random
import time

import pandas as pd
import requests
from lxml import etree

# 真实cookie
Cookie = 'user_trace_token=20190312230635-bd10a4e8-55d7-48b5-962d-f9227b07ccd4; _ga=GA1.2.434082682.1552403204; LGUID=20190312230642-71a19b72-44d8-11e9-946e-5254005c3644; JSESSIONID=ABAAABAAAIAACBIDC1DB26FED5F93A2A2B73CA1A5C421FD; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557148390; LGSID=20190506211309-b17281e3-7000-11e9-8664-525400f775ce; PRE_UTM=; PRE_HOST=; PRE_SITE=; PRE_LAND=https%3A%2F%2Fwww.lagou.com%2F; index_location_city=%E5%85%A8%E5%9B%BD; _gid=GA1.2.1330006640.1557148395; TG-TRACK-CODE=index_search; _gat=1; LG_LOGIN_USER_ID=c6ffaba7ee92122f4b041f9a849d4f9d924b7edf620798a600169b4fff1a9f6d; LG_HAS_LOGIN=1; _putrc=5F44CFCCC08C9A2A123F89F2B170EADC; login=true; unick=%E9%AD%8F%E5%85%83%E5%AE%9D; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=33; gate_login_token=9906db5b022596f1e3c42a1a991e55e88620b336119a01db8145d3c90d65c1be; SEARCH_ID=ec66f5ad9f234e27a58c0459542bff8e; X_HTTP_TOKEN=d3a07b50a9f6063e8269417551a0b095c1f90e316f; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557149630; LGRID=20190506213348-94402a8b-7003-11e9-8665-525400f775ce'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36',
#     'Cookie': Cookie
}


for i in range(1, 3):
    time.sleep(random.randint(3, 10))
    url = 'https://www.lagou.com/zhaopin/jiqixuexi/{}/?filterOption=2'.format(i)
    print('正在抓取第{}页数据...'.format(i))

    # 请求网页并解析
    con = etree.HTML(requests.get(url, headers=headers).text)

    # 使用xpath表达式抽取各个字段信息
    job_name = [i for i in con.xpath('//a[@class="position_link"]/h3/text()')]
    job_address = [i for i in con.xpath("//a[@class='position_link']/span/em/text()")]
    job_company = [i for i in con.xpath("//div[@class='company_name']/a/text()")]
    job_salary = [i for i in con.xpath("//span[@class='money']/text()")]
    job_exp_edu = [i for i in con.xpath("//div[@class='li_b_l']/text()")]
    job_exp_edu2 = [i for i in [i.strip() for i in job_exp_edu] if i != '']
    job_industry = [i for i in con.xpath("//div[@class='industry']/text()")]
    job_tempation = [i for i in con.xpath("//div[@class='list_item_bot']/div[@class='li_b_r']/text()")]
    job_links = [i for i in con.xpath("//div[@class='p_top']/a/@href")]

    # 获取详情页链接后采集详情页岗位描述信息
    job_des = []
    for link in job_links:
        time.sleep(random.randint(3, 10))
        con2 = etree.HTML(requests.get(url=link, headers=headers).text)
        des = [[i.xpath('string(.)') for i in con2.xpath('//dd[@class="job_bt"]/div/p')]]

        job_des += des
    break

# 对数据进行字典封装
dataset = {
    '岗位名称': job_name,
    '工作地址': job_address,
    '公司': job_company,
    '薪资': job_salary,
    '经验学历': job_exp_edu2,
    '所属行业': job_industry,
    '岗位福利': job_tempation,
    '任职要求': job_des
}

# 转化为数据框并存为csv
data = pd.DataFrame(dataset)
data.to_csv('machine_learning_hz_job1.csv')

正在抓取第1页数据...


In [12]:
data.head()

,岗位名称,工作地址,公司,薪资,经验学历,所属行业,岗位福利,任职要求
0,深度/机器学习,北京·中关村,深睿医疗,25k-50k,经验不限 / 硕士,"\n 移动互联网,医疗健康 /...","“大牛,人性化管理,靠谱团队,技术氛围好”",[研究深度学习（包括各种神经网络结构与应用）或计算机视觉各个领域（目标检测识别等）中核心算法...
1,机器学习,北京·西直门,Udesk－企业级智能客服平台,15k-30k,经验1-3年 / 硕士,"\n 移动互联网,企业服务 /...","“团队强悍,待遇丰厚,数据规模,场景明确”","[职位描述：, + 企业服务领域智能化场景的业务分析、数据建模和数据优化。, , , 任职要..."
2,头条实验室算法工程师-机器学习,北京·海淀区,字节跳动,25k-50k,经验1-3年 / 本科,"\n 移动互联网,数据服务 /...",“六险一金，免费三餐，租房补贴，带薪休假”,[]
3,机器学习专家,杭州·东新路,点我达,30k-60k,经验5-10年 / 硕士,\n O2O / D轮及以上 ...,“独角兽公司、股票期权、高速发展”,"[岗位职责：, 点我达的智能调度系统中包含大量机器学习的问题，包括订单压力预测、实时单量预测..."
4,机器学习研发工程师,成都·高新区,Camera360,15k-25k,经验1-3年 / 硕士,\n 移动互联网 / B轮 /...,"“五险一金,员工体检,职级晋升,股票期权”",[岗位职责：1.深度学习方向相关前沿技术的研发。任职要求：1.重点大学计算机相关专业研究生以...


In [15]:
import json
import random
import time
import requests
from bs4 import BeautifulSoup
import pandas as pd

#定义抓取主函数
def lagou_dynamic_crawl():
    headers = {
        'Accept': 'application/json, text/javascript, */*; q=0.01',
        'Referer': 'https://www.lagou.com/jobs/list_%E8%BF%90%E7%BB%B4?city=%E6%88%90%E9%83%BD&cl=false&fromSearch=true&labelWords=&suginput=',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36'
    }

    # 创建一个职位列表容器
    positions = []
    for page in range(1, 31):
        print('正在抓取{}页数据...'.format(page))
        URL_ = 'https://www.lagou.com/jobs/list_%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0?city=%E5%85%A8%E5%9B%BD&cl=false&fromSearch=true&labelWords=&suginput='
        URL = 'https://www.lagou.com/jobs/positionAjax.json?needAddtionalResult=false'
        # 构建请求保单参数
        params = {
            'first': 'false',
            'pn': page,
            'kd': '机器学习'
        }

        # 构造请求并返回结果
        s = requests.Session()
        s.get(URL_, headers=headers, timeout=3)  # 请求首页获取cookies
        cookie = s.cookies  # 为此次获取的cookies
        result = s.post(URL, headers=headers, data=params, cookies=cookie, timeout=3)
#         print(result.text)

        # 将请求结果转化为json
        json_result = result.json()

        # 解析json数据获取目标信息
        try:
            position_info = json_result['content']['positionResult']['result']
        except:
            position_info = json_result['msg']
            if '您操作太频繁' in position_info:
                print('操作太频繁')
                assert False

        # 循环当前页每一个职位信息，再去爬职位详情页面
        for position in position_info:
            # 把我们要爬取的信息放入字典
            position_dict = {
                'position_name': position['positionName'],
                'work_year': position['workYear'],
                'education': position['education'],
                'salary': position['salary'],
                'city': position['city'],
                'company_name': position['companyFullName'],
                'address': position['businessZones'],
                'label': position['companyLabelList'],
                'stage': position['financeStage'],
                'size': position['companySize'],
                'advantage': position['positionAdvantage'],
                'industry': position['industryField'],
                'industryLables': position['industryLables']
            }
            # 找到职位id
            position_id = position['positionId']

            # 根据职位id调用岗位描述函数获取职位描述

            position_dict['position_detail'] = recruit_detail(position_id)
            positions.append(position_dict)

        time.sleep(random.randint(3, 6))
    print('全部数据采集完毕...')
    return positions

# 定义抓取岗位描述函数
def recruit_detail(position_id):
    headers = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Cookie": "user_trace_token=20190312230635-bd10a4e8-55d7-48b5-962d-f9227b07ccd4; _ga=GA1.2.434082682.1552403204; LGUID=20190312230642-71a19b72-44d8-11e9-946e-5254005c3644; JSESSIONID=ABAAABAAAIAACBIDC1DB26FED5F93A2A2B73CA1A5C421FD; Hm_lvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557148390; index_location_city=%E5%85%A8%E5%9B%BD; _gid=GA1.2.1330006640.1557148395; LG_LOGIN_USER_ID=c6ffaba7ee92122f4b041f9a849d4f9d924b7edf620798a600169b4fff1a9f6d; LG_HAS_LOGIN=1; _putrc=5F44CFCCC08C9A2A123F89F2B170EADC; login=true; unick=%E9%AD%8F%E5%85%83%E5%AE%9D; showExpriedIndex=1; showExpriedCompanyHome=1; showExpriedMyPublish=1; hasDeliver=33; gate_login_token=9906db5b022596f1e3c42a1a991e55e88620b336119a01db8145d3c90d65c1be; X_MIDDLE_TOKEN=411fdb1739a24c8cf116bd0c0a582ee8; TG-TRACK-CODE=search_code; SEARCH_ID=edea9ba44bc9459db8eb58ec279ca51b; X_HTTP_TOKEN=d3a07b50a9f6063e2803517551a0b095c1f90e316f; Hm_lpvt_4233e74dff0ae5bd0a3d81c6ccf756e6=1557153083; LGRID=20190506223122-9eb78021-700b-11e9-8688-525400f775ce",
        "Host": "www.lagou.com",
        "Referer": "https://www.lagou.com/jobs/list_%E6%9C%BA%E5%99%A8%E5%AD%A6%E4%B9%A0?labelWords=&fromSearch=true&suginput=",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36",
    }
    url = 'https://www.lagou.com/jobs/%s.html' % position_id
    result = requests.get(url, headers=headers)
    time.sleep(random.randint(1, 4))

    # 解析职位要求text
    soup = BeautifulSoup(result.text, 'html.parser')
    job_bt = soup.find(class_='job_bt')

    # 通过尝试发现部分记录描述存在空的情况
    # 所以这里需要判断处理一下
    if job_bt != None:
        job_bt = job_bt.text
    else:
        job_bt = 'null'
    return job_bt


if __name__ == '__main__':
    positions = lagou_dynamic_crawl()
    data = pd.DataFrame(positions)
    data.to_csv('machine_learning_hz_job2.csv')

正在抓取1页数据...
正在抓取2页数据...
正在抓取3页数据...
正在抓取4页数据...
正在抓取5页数据...
正在抓取6页数据...
正在抓取7页数据...
正在抓取8页数据...
正在抓取9页数据...
正在抓取10页数据...
正在抓取11页数据...
正在抓取12页数据...
正在抓取13页数据...
正在抓取14页数据...
正在抓取15页数据...
正在抓取16页数据...
正在抓取17页数据...
正在抓取18页数据...
正在抓取19页数据...
正在抓取20页数据...
正在抓取21页数据...
正在抓取22页数据...
正在抓取23页数据...
正在抓取24页数据...
正在抓取25页数据...
正在抓取26页数据...
正在抓取27页数据...
正在抓取28页数据...
正在抓取29页数据...
正在抓取30页数据...
全部数据采集完毕...


In [16]:
positions

[{'position_name': '深度/机器学习',
  'work_year': '不限',
  'education': '硕士',
  'salary': '25k-50k',
  'city': '北京',
  'company_name': '北京深睿博联科技有限责任公司',
  'address': ['中关村', '知春路', '苏州街'],
  'label': ['股票期权', '年底双薪', '定期体检', '弹性工作'],
  'stage': 'B轮',
  'size': '150-500人',
  'advantage': '大牛,人性化管理,靠谱团队,技术氛围好',
  'industry': '移动互联网,医疗健康',
  'industryLables': ['医疗健康', '机器学习', '深度学习', '计算机视觉', '算法'],
  'position_detail': 'null'},
 {'position_name': '机器学习',
  'work_year': '1-3年',
  'education': '硕士',
  'salary': '15k-30k',
  'city': '北京',
  'company_name': '北京沃丰时代数据科技有限公司',
  'address': ['西直门', '动物园', '官园'],
  'label': ['AI', '大数据', '云服务', '大牛带队'],
  'stage': 'C轮',
  'size': '150-500人',
  'advantage': '团队强悍,待遇丰厚,数据规模,场景明确',
  'industry': '移动互联网,企业服务',
  'industryLables': ['企业服务', '搜索', 'NLP', '算法'],
  'position_detail': 'null'},
 {'position_name': '头条实验室算法工程师-机器学习',
  'work_year': '1-3年',
  'education': '本科',
  'salary': '25k-50k',
  'city': '北京',
  'company_name': '北京字节跳动科技有限公司',
  'address': No

In [17]:
df = pd.DataFrame(positions)
df.shape

(450, 14)

In [1]:
df.head()

NameError: name 'df' is not defined